In [4]:
# for manipulating data
import pandas as pd
# setting the random seed for reproducibility
import random
random.seed(493)
# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\jesud\miniconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


In [5]:
df = pd.read_csv('trump_20200530_clean.csv', parse_dates=True, index_col='datetime')

In [6]:
df.head()
df.info()

,tweet,retweets,favorites
datetime,,,
2020-05-30 03:20:18,Time for a change! #2020 https://t.co/AECy2GBfys,42879,164022
2020-05-29 19:46:15,https://t.co/mljmx2o0G7,48713,176310
2020-05-29 18:20:06,Looting leads to shooting and that’s why a man was shot and killed in Minneapolis on Wednesday night - or look at what just happened in Louisville with 7 people shot. I don’t want this to happen and that’s what the expression put out last night means....,52444,241219
2020-05-29 18:20:06,....It was spoken as a fact not as a statement. It’s very simple nobody should have any problem with this other than the haters and those looking to cause trouble on social media. Honor the memory of George Floyd!,31263,158594
2020-05-29 15:15:41,REVOKE 230!,38582,195262


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15890 entries, 2020-05-30 03:20:18 to 2017-01-20 17:51:25
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      15890 non-null  object
 1   retweets   15890 non-null  int64 
 2   favorites  15890 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 496.6+ KB


In [7]:
# for natural language processing: named entity recognition
import spacy
import en_core_web_sm

In [8]:
#We’re only going to use spaCy’s ner functionality or named-entity recognition so we’ll disable the rest of the functionalities. This will save us a lot of loading time later
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'textcat'])

In [9]:
#Now let’s load the contents stopwords file into the variable stopswords. Note that we converted the list into a set to also save some processing time later.
with open('twitter-stopwords - TA - Less.txt') as f:
    contents = f.read().split(',')
stopwords = set(contents)

##### Next, we’ll import joblib and define a few functions to help with parallel processing.

In [13]:
from joblib import Parallel, delayed
def chunker(iterable, total_length, chunksize):
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

In [16]:
def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append([(ent.text) for ent in doc.ents if ent.label_ in ['NORP', 'PERSON', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT']])
    return preproc_pipe

In [11]:
def preprocess_parallel(texts, chunksize=1000):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, len(df), chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)

In [ ]:
df['entities'] = preprocess_parallel(df['tweet'], chunksize=1000)

In [ ]:
entities = [entity for entity in df.entities if entity != []]
entities = [item for sublist in entities for item in sublist]
entities_set = set(entities)

In [ ]:
df.head()

In [ ]:
entities = [entity for entity in df.entities if entity != []]
entities = [item for sublist in entities for item in sublist]
entities_set = set(entities)

In [ ]:
df_counts = pd.Series(entities).value_counts()[:20].to_frame().reset_index()
df_counts.columns=['entity', 'count']
df_counts

In [ ]:
from spacy.lang.en import English

In [ ]:
nlp = English()

In [ ]:
doc1 = nlp("I like fast food")
doc2 = nlp("I like Pizza")
print(doc1.similarity(doc2))